In [1]:
import pyomo.environ as pyo

In [2]:
NODES = ['DB', 'KB', 'KW', 'SW', 'GW', 'DM', 'GM', 'CM', 'DUMMY']
WAREHOUSES = ['KW','SW','GW']
ARCS = [('DB','DM'),('DB','KW'),('DB','SW'),('DB','GW'),('DB','DUMMY'),
        ('KB','KW'),('KB','SW'),('KB','GW'),('KB','DUMMY'),
        ('KW','DM'),('KW','GM'),('KW','CM'),
        ('SW','DM'),('SW','GM'),('SW','CM'),
        ('GW','DM'),('GW','GM'),('GW','CM')]

# Parameters
B = {'DB':700, 'KB':800, 'KW':0, 'SW':0, 'GW':0,
     'DM':-600, 'GM':-500, 'CM':-300, 'DUMMY':-100}


COST = {('DB','DM'):18,('DB','KW'):15,('DB','SW'):20,('DB','GW'):15,('DB','DUMMY'):0,
        ('KB','KW'):10,('KB','SW'):25,('KB','GW'):20,('KB','DUMMY'):0,
        ('KW','DM'):16,('KW','GM'):12,('KW','CM'):11,
        ('SW','DM'):21,('SW','GM'):9,('SW','CM'):28,
        ('GW','DM'):16,('GW','GM'):5,('GW','CM'):12}
RENT = {'KW':240, 'SW':450, 'GW':320}
CAPACITY = {'KW':400, 'SW':800,'GW':600}

In [3]:
 model = pyo.ConcreteModel()
    
    # Decision variables
model.x = pyo.Var(ARCS, domain=pyo.NonNegativeIntegers)
model.z = pyo.Var(WAREHOUSES, domain=pyo.Binary)
    

In [4]:
# Objective function
def obj_rule(model):
    return (sum(COST[i,j]*model.x[i,j] for i,j in ARCS) +
            sum(RENT[w]*model.z[w] for w in WAREHOUSES))
model.obj = pyo.Objective(rule=obj_rule)

In [5]:
# Balance of flow
def balance_rule(model, node):
    return (sum(model.x[i,j] for i,j in ARCS if i==node) 
            - sum(model.x[i,j] for i,j in ARCS if j==node)
            == B[node])
model.balance_constraint = pyo.Constraint(NODES,rule=balance_rule)

In [6]:
# Weak fixed charge & warehouse capacity
def weak_rule(model, wh):
    return (sum(model.x[i,j] for i,j in ARCS if j==wh)
            <= CAPACITY[wh]*model.z[wh])
model.weak_constraint = pyo.Constraint(WAREHOUSES,rule=weak_rule)

In [7]:
# solve model
solver_result = pyo.SolverFactory('glpk').solve(model)

# Check if the model solved to optimality before printing solution
solve_status = solver_result.solver.termination_condition
if (solve_status==pyo.TerminationCondition.optimal):
    print(f'Min cose (transport + rent) is {model.obj()}\n')
    for w in WAREHOUSES:
        if model.z[w]==0:
            print(f'Warehouse {w} is not used')
        else:
            print(f'Warehouse {w} is used') 
    print('')
    print('Cases shipped:')
    for (i,j) in ARCS:
        if model.x[i,j].value != 0:
           print(f'{i} to {j}: {model.x[i,j].value}')
    print('\nAll other flows are 0')
else:
    print(f'The solver status is {solve_status}')

Min cose (transport + rent) is 29360.0

Warehouse KW is used
Warehouse SW is not used
Warehouse GW is used

Cases shipped:
DB to DM: 600.0
DB to GW: 100.0
KB to KW: 400.0
KB to GW: 300.0
KB to DUMMY: 100.0
KW to GM: 100.0
KW to CM: 300.0
GW to GM: 400.0

All other flows are 0
